<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun May 29 21:27:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')


import os 
path_drive = '/content/gdrive/MyDrive/Colab Notebooks/TrackNet-Padel/'
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-SoloPadel'

!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/TrackNet-Padel


In [3]:
os.chdir('/content/')
!pwd

/content


In [4]:
!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull

Cloning into 'TrackNet-Padel'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 597 (delta 6), reused 30 (delta 5), pack-reused 565
Receiving objects: 100% (597/597), 666.67 MiB | 42.53 MiB/s, done.
Resolving deltas: 100% (274/274), done.
Checking out files: 100% (139/139), done.
Already up to date.


In [ ]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

### Test accuracy

In [ ]:
file_name = 'Final_PremierQatar_Tie'
point_n = 0
for point_n in range(9,13):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".resPadel.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} #--load_weights={weights_path+'4'} # 2>&1 | tee {res}

raw_data/points/Final_PremierQatar_Tie_9.mp4
2022-05-25 21:43:20.779225: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......


### Train Padel

In [6]:
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

raw_data/points/*_0\.*
raw_data/points/*_1\.*
raw_data/points/*_2\.*
raw_data/points/*_3\.*
raw_data/points/*_4\.*
raw_data/points/*_5\.*
raw_data/points/*_6\.*
raw_data/points/*_7\.*
raw_data/points/*_8\.*


In [ ]:
!rm -rf train_data/*
!python video2img.py 

In [15]:
!python train.py --save_weights={weights_path} --pre_trained=True --batch_size=5 --load_weights={weights_path}

lerning rate: 2.0
2022-05-29 21:36:19.015392: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Beginning training......
==========Epoch 0, Train steps: 3477, Learning rate: 1.0000==========
3477/3477 [==============================] - 1370s 391ms/step - loss: 3.4314e-04 - binary_accuracy: 0.9996
==========Epoch 0 start validation==========
Epoch 0 accuracy: 0.648
Epoch 0 precision: 0.870
Epoch 0 recall: 0.635
Epoch 0 average = (accuracy + precision + recall)/3: 0.718
==========Epoch 1, Train steps: 3477, Learning rate: 1.0000==========
3477/3477 [==============================] - 1357s 390ms/step - loss: 2.9414e-04 - binary_accuracy: 0.9996
==========Epoch 1 start validation==========
Epoch 1 accuracy: 0.648
Epoch 1 precision: 0.878
Epoch 1 recall: 0.615
Epoch 1 average = (accuracy + precision + recall)/3: 0.714
==========Epoch 2, Train steps: 347